## SRGAN Implementation
For deep learning course

In [1]:
import argparse
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

import pytorch_ssim
from data_utils import TrainDatasetFromFolder, ValDatasetFromFolder, display_transform
#from loss import GeneratorLoss
from model import Generator, Discriminator

In [2]:
import torch
from torch import nn
from torchvision.models.vgg import vgg16


class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg16(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss
        perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss


class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]


#if __name__ == "__main__":
#    g_loss = GeneratorLoss()
#    print(g_loss)

In [3]:
# Network Training option
# =======================
# CROPS_SIZE = Training images crop size
# UPSCALE_FACTOR = Super resolution scaling
# NUM_EPOCHS = Number of training epoch

CROP_SIZE = int(64)
UPSCALE_FACTOR = int(4)
NUM_EPOCHS = int(50)

netG = Generator(UPSCALE_FACTOR)
print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
netD = Discriminator()
print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))

generator_criterion = GeneratorLoss()

# Netowrk optimizer
optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

if torch.cuda.is_available():
    netG.cuda()
    netD.cuda()
    generator_criterion.cuda()

# generator parameters: 734219
# discriminator parameters: 5215425


In [4]:
# Load trained model
print ('Load trained model')
netG.load_state_dict(torch.load('network/netG_epoch_4_50_animeDS.pth'))
netD.load_state_dict(torch.load('network/netD_epoch_4_50_animeDS.pth'))
netG.eval()
netD.eval()

Load trained model


Discriminator(
  (net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.2)
    (14): Conv2d(256, 256, 

#### Dataset Folder


In [5]:
train_set = TrainDatasetFromFolder('data/anime_train_HR', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set = ValDatasetFromFolder('data/anime_valid_HR', upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)
print("Dataset loaded")

Dataset loaded


#### Network Architecture
Network architecutre is defined in **model.py**

It includes:
- Generator(Residual Network)
- Discriminator(VGG Network)

#### Dataset Pre processing
Pre-processing is defined in **data_utils.py**

It includes:
- Image transform 
- Dataset processing

#### Loss Function
Loss functions for training the network

#### Network Training
Specify training option

In [ ]:
results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

for epoch in range(1, NUM_EPOCHS + 1):
    train_bar = (train_loader)
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

    netG.train()
    netD.train()
    print("Epoch Number: %d" %(epoch))
    for data, target in train_bar:
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target)
        if torch.cuda.is_available():
            real_img = real_img.cuda()
        z = Variable(data)
        if torch.cuda.is_available():
            z = z.cuda()
        fake_img = netG(z)

        netD.zero_grad()
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()


        optimizerG.step()

        # loss for current batch before optimization
        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size
        
        

   
        print("[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f" % (
            epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes'])) 

    # save model parameters
    torch.save(netG.state_dict(), 'epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch+50))
    torch.save(netD.state_dict(), 'epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch+50))
    


Epoch Number: 1
[1/50] Loss_D: 1.0000 Loss_G: 0.0039 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0070 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0066 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0072 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0072 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0070 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0070 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0069 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0069 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0067 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0066 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0065 D(x): 0.0000 D(G(z)): 0

[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0057 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Los

[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Loss_D: 1.0000 Loss_G: 0.0056 D(x): 0.0000 D(G(z)): 0.0000
[1/50] Los

[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Los

[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0054 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0054 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Loss_D: 1.0000 Loss_G: 0.0055 D(x): 0.0000 D(G(z)): 0.0000
[2/50] Los

...

[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Loss_D: 1.0000 Loss_G: 0.0045 D(x): 1.0000 D(G(z)): 1.0000
[9/50] Los

[10/50] Loss_D: 1.0000 Loss_G: 0.0048 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0048 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0048 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0049 D(x): 1.0000 D(G(z)): 1.

[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0046 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0047 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0047 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0047 D(x): 1.0000 D(G(z)): 1.

[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.0000
[10/50] Loss_D: 1.0000 Loss_G: 0.0068 D(x): 1.0000 D(G(z)): 1.

[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0060 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.0000
[11/50] Loss_D: 1.0000 Loss_G: 0.0059 D(x): 1.0000 D(G(z)): 1.

#### Save training result

In [ ]:



        train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f' % (
            epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']))
    
    
    
parser = argparse.ArgumentParser(description='Train Super Resolution Models')
parser.add_argument('--crop_size', default=88, type=int, help='training images crop size')
parser.add_argument('--upscale_factor', default=4, type=int, choices=[2, 4, 8],
                    help='super resolution upscale factor')
parser.add_argument('--num_epochs', default=100, type=int, help='train epoch number')

# argparse is a command to parse argument from user. In this program,it is used to specify some parameter
# configuration before starting the network training

# __name__ == '__main__' is used to make sure the program is run as a main program. If it is called by other,
# it will not run the code below.
#if __name__ == '__main__':
#opt = parser.parse_args()

#CROP_SIZE = opt.crop_size
#UPSCALE_FACTOR = opt.upscale_factor
#NUM_EPOCHS = opt.num_epochs




    netG.eval()
    out_path = 'training_results/SRF_' + str(UPSCALE_FACTOR) + '/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    with torch.no_grad():
        val_bar = tqdm(val_loader)
        valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
        val_images = []
        for val_lr, val_hr_restore, val_hr in val_bar:
            batch_size = val_lr.size(0)
            valing_results['batch_sizes'] += batch_size
            lr = val_lr
            hr = val_hr
            if torch.cuda.is_available():
                lr = lr.cuda()
                hr = hr.cuda()
            sr = netG(lr)

            batch_mse = ((sr - hr) ** 2).data.mean()
            valing_results['mse'] += batch_mse * batch_size
            batch_ssim = pytorch_ssim.ssim(sr, hr).item()
            valing_results['ssims'] += batch_ssim * batch_size
            valing_results['psnr'] = 10 * log10(1 / (valing_results['mse'] / valing_results['batch_sizes']))
            valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
            val_bar.set_description(
                desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                    valing_results['psnr'], valing_results['ssim']))

            val_images.extend(
                [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                 display_transform()(sr.data.cpu().squeeze(0))])
        val_images = torch.stack(val_images)
        val_images = torch.chunk(val_images, val_images.size(0) // 15)
        #val_save_bar = tqdm(val_images, desc='[saving training results]')
        #index = 1
        #for image in val_save_bar:
        #    image = utils.make_grid(image, nrow=3, padding=5)
        #    utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
        #    index += 1
# save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(valing_results['psnr'])
    results['ssim'].append(valing_results['ssim'])

    if epoch % 10 == 0 and epoch != 0:
        out_path = 'statistics/'
        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')

#### Network Testing

In [ ]:
from scipy.misc import imread, imsave, imresize
image = imread(f_name)
if(len(image.shape)<3):
      print ('gray')
elif len(image.shape)==3:
      print ('Color(RGB)')
else:
      print ('others')

In [5]:
import matplotlib.pyplot as plt
val_set
fig = plt.figure()

for lr,sr,hr,loc in val_set:
    imSize = hr.size()
    
    print(loc, imSize)

    if loc == 3:
        #plt.show()
        break

0 torch.Size([3, 512, 512])
1 torch.Size([3, 512, 512])
2 torch.Size([3, 512, 512])
3 torch.Size([3, 512, 512])


In [12]:
import matplotlib.pyplot as plt
from PIL import Image
idx =[]
len(val_set)
for lr,sr,hr,loc in val_set:
    imSize = hr.size()
    if imSize[0] ==1:
        #print(imSize[0])
        #print(loc)
        idx.append(loc)
        #break
    if loc % 100 == 0 and loc != 0:
        print(loc)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


In [13]:
print(idx)
len(idx)

[]


0

In [9]:
import os
for i in range (len(idx)):
    
    imLoc = idx[i]
    #print(train_set.image_filenames[imLoc])
    os.remove(val_set.image_filenames[imLoc])
    print(i)
    #if i >20:
    #    break
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215


In [ ]:
len(idx)